In [ ]:
import json, os
import pandas as pd
from matplotlib import pyplot as plt
from collections import defaultdict
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.decomposition import PCA
# from wpca import WPCA
from sklearn.preprocessing import StandardScaler
from aquabyte.accuracy_metrics import AccuracyMetricsGenerator
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
from aquabyte.visualize import Visualizer, _normalize_world_keypoints
from aquabyte.template_matching import find_matches_and_homography
import random
import pickle
import cv2
from copy import copy
from scipy.stats import norm
from mpl_toolkits.mplot3d import Axes3D
from multiprocessing import Pool, Manager
import matplotlib.cm as cm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

<h1> Extract base data from database </h1>

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_RESEARCH_SQL_CREDENTIALS'])))
query = """
    select * from research.fish_metadata a left join keypoint_annotations b
    on a.left_url = b.left_image_url 
    where b.keypoints is not null and b.is_qa = false;
"""
df = rds_access_utils.extract_from_database(query)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')


<h1> Generate Sample Template Matching Results </h1>

In [ ]:
from copy import copy
import datetime
import json, os

import boto3
import cv2
import numpy as np
from collections import namedtuple
import pandas as pd
import psycopg2.extras
import psycopg2

from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.optics import convert_to_world_point, depth_from_disp, pixel2world, euclidean_distance


def enhance(image, clip_limit=5):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert image from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image

def crop_images(imageL, imageR, left_crop_metadata, right_crop_metadata, left_keypoints_dict=None, 
    right_keypoints_dict=None, PADDING=100):
    
    # crop the data
    l_width = left_crop_metadata['width']
    l_height = left_crop_metadata['height']
    r_width = right_crop_metadata['width']
    r_height = right_crop_metadata['height']
    
    cropL_x_left = max(min([kp[0] for kp in left_keypoints_dict.values()]) - PADDING, 0)
    cropL_x_right = min(max([kp[0] for kp in left_keypoints_dict.values()]) + PADDING, l_width)
    cropL_y_top = max(min([kp[1] for kp in left_keypoints_dict.values()]) - PADDING, 0)
    cropL_y_bottom = min(max([kp[1] for kp in left_keypoints_dict.values()]) + PADDING, l_height)

    cropR_x_left = max(min([kp[0] for kp in right_keypoints_dict.values()]) - PADDING, 0)
    cropR_x_right = min(max([kp[0] for kp in right_keypoints_dict.values()]) + PADDING, r_width)
    cropR_y_top = max(min([kp[1] for kp in right_keypoints_dict.values()]) - PADDING, 0)
    cropR_y_bottom = min(max([kp[1] for kp in right_keypoints_dict.values()]) + PADDING, r_height)

    imageL = imageL[cropL_y_top:cropL_y_bottom, cropL_x_left:cropL_x_right]
    imageR = imageR[cropR_y_top:cropR_y_bottom, cropR_x_left:cropR_x_right]

    return imageL, imageR, cropL_x_left, cropL_y_top, cropR_x_left, cropR_y_top


def get_body_keypoints(kp1, kp2, matches, matchesMask, left_crop_metadata, right_crop_metadata): 
    kps = []
    for i, m in enumerate(matches):
        # Generate random color for RGB/BGR and grayscale images as needed.            
        if matchesMask[i] == 1:
            p1 = tuple(np.round(kp1[m.queryIdx].pt).astype(float))
            p2 = tuple(np.round(kp2[m.trainIdx].pt).astype(float))
            p1_x_frame = p1[0] + left_crop_metadata['x_coord']
            p1_y_frame = p1[1] + left_crop_metadata['y_coord']
            p2_x_frame = p2[0] + right_crop_metadata['x_coord']
            p2_y_frame = p2[1] + right_crop_metadata['y_coord']
            disp = abs(p1_x_frame - p2_x_frame)
            kp = [p1_x_frame, p1_y_frame, p2_x_frame, p2_y_frame]
            kps.append(kp)
    
    return kps

def get_modified_crop_metadata(left_crop_metadata, right_crop_metadata, cropL_x_left, cropL_y_top, cropR_x_left, cropR_y_top):
    modified_left_crop_metadata = {
        'x_coord': left_crop_metadata['x_coord'] + cropL_x_left,
        'y_coord': left_crop_metadata['y_coord'] + cropL_y_top
    }

    modified_right_crop_metadata = {
        'x_coord': right_crop_metadata['x_coord'] + cropR_x_left,
        'y_coord': right_crop_metadata['y_coord'] + cropR_y_top
    }

    return modified_left_crop_metadata, modified_right_crop_metadata



def find_matches_and_homography(imageL, imageR, cm, left_crop_metadata, right_crop_metadata, keypoints=None,
    MIN_MATCH_COUNT=11, GOOD_PERC=0.7, FLANN_INDEX_KDTREE=0):

    if keypoints is not None:
        if 'leftCrop' in keypoints and 'rightCrop' in keypoints:
            left_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['leftCrop']}
            right_keypoints_dict = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in keypoints['rightCrop']}
            
            # crop images to speed up the template matching process
            imageL, imageR, cropL_x_left, cropL_y_top, cropR_x_left, cropR_y_top = \
                crop_images(imageL, imageR, left_crop_metadata, right_crop_metadata, left_keypoints_dict, right_keypoints_dict)

            modified_left_crop_metadata, modified_right_crop_metadata = \
                get_modified_crop_metadata(left_crop_metadata, right_crop_metadata, cropL_x_left, cropL_y_top, cropR_x_left, cropR_y_top)

        print(left_crop_metadata)
        print(modified_left_crop_metadata)
    else:
        modified_left_crop_metadata, modified_right_crop_metadata = left_crop_metadata, right_crop_metadata

    
    # find template matches
    sift = cv2.KAZE_create()
    img1 = enhance(imageL)
    img2 = enhance(imageR)
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1,des2, k=2)
    good = []
    matchesMask = []
    
    for m,n in matches:
        if m.distance < GOOD_PERC*n.distance:
            good.append(m)
    if len(good)>=MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1, 1 ,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1, 1, 2)
        H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()
        if H is None:
            return [[[]]]
        H = H.tolist()
        kps = get_body_keypoints(kp1, kp2, good, matchesMask, modified_left_crop_metadata, modified_right_crop_metadata)
        return [H, kps]

    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
        return [[[]]]

    return [[[]]]


def get_wkps(kps, cm): 
    wkps = []
    for kp in kps:
        p1_x_frame, p1_y_frame, p2_x_frame, p2_y_frame = kp
        disp = abs(p1_x_frame - p2_x_frame)
        depth = depth_from_disp(disp, cm)
        wkp = convert_to_world_point(p1_x_frame, p1_y_frame, depth, cm)
        wkps.append(list(wkp))
        
    return wkps


def determine_fish_distance(kps, cm):    
    wkps = get_wkps(kps, cm)
    dist = np.median([wkp[1] for wkp in wkps])
    return dist


def main():
    s3_access_utils = S3AccessUtils('/root/data')
    rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
    
    query = """
        SELECT * FROM prod.crop_annotation ca
        INNER JOIN prod.annotation_state pas on pas.id=ca.annotation_state_id
        WHERE ca.service_id = (SELECT ID FROM prod.service where name='LATI')
        AND ca.left_crop_url is not null
        AND ca.right_crop_url is not null
        AND ca.pen_id = 64
        AND (ca.annotation_state_id=6 OR ca.annotation_state_id=7)
        AND ca.captured_at > '2019-09-01'
        LIMIT 2;
    """

    df = rds_access_utils.extract_from_database(query)
    distances_from_camera = []
    for idx, row in df.iterrows():
        left_image_url, right_image_url = row.left_crop_url, row.right_crop_url
        left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
        keypoints = row.keypoints
        cm = row.camera_metadata

        left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
        right_image_f, _, _ = s3_access_utils.download_from_url(right_image_url)
        imageL = cv2.imread(left_image_f)
        imageR = cv2.imread(right_image_f)
        H, kps = find_matches_and_homography(imageL, imageR, cm, left_crop_metadata, right_crop_metadata, keypoints)
        dist = determine_fish_distance(kps, cm)
        distances_from_camera.append(dist)
    df['distance_from_camera'] = distances_from_camera


    







In [ ]:
idx = 0
row = df[~df.left_image_url.str.contains('aquabyte-crops')].iloc[idx]
left_image_url = row.left_image_url
right_image_url = row.right_image_url
keypoints = row.keypoints
# left_image_url, right_image_url = df.left_image_url.iloc[idx], df.right_image_url.iloc[idx]
left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
cm = row.camera_metadata
keypoints = row.keypoints

left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
right_image_f, _, _ = s3_access_utils.download_from_url(right_image_url)
imageL = cv2.imread(left_image_f)
imageR = cv2.imread(right_image_f)
H, kps = find_matches_and_homography(imageL, imageR, cm, left_crop_metadata, right_crop_metadata, keypoints)

In [ ]:
plt.scatter([item['xFrame'] for item in keypoints['leftCrop']], [item['yFrame'] for item in keypoints['leftCrop']], color='blue')
plt.scatter(np.array(kps)[:, 0], np.array(kps)[:, 1], color='red')
plt.show()

In [ ]:
row = df.iloc[idx]
keypoints = row.keypoints
plt.scatter([item['xFrame'] for item in keypoints['rightCrop']], [item['yFrame'] for item in keypoints['rightCrop']], color='blue')
plt.scatter(np.array(kps)[:, 2], np.array(kps)[:, 3], color='red')
plt.show()

In [ ]:
left_image_url

In [ ]:
def get_wkps(kps, cm): 
    # Draw lines between matches.  Make sure to offset kp coords in second image appropriately.
    wkps = []
    for kp in kps:
        p1_x_frame, p1_y_frame, p2_x_frame, p2_y_frame = kp
        disp = abs(p1_x_frame - p2_x_frame)
        depth = depth_from_disp(disp, cm)
        wkp = convert_to_world_point(p1_x_frame, p1_y_frame, depth, cm)
        wkps.append(list(wkp))
        
    return wkps


In [ ]:
body_wkps = np.array(get_wkps(kps, cm))

<h1> Visualize World Keypoints </h1>

In [ ]:
# create local util functions for body world keypoint visualizations (will override original imports)

def _generate_rotation_matrix(u_base, v):
    u = v / np.linalg.norm(v)
    n = np.cross(u_base, u)
    n = n / np.linalg.norm(n)
    theta = -np.arccos(np.dot(u, u_base))

    R = np.array([[
        np.cos(theta) + n[0]**2*(1-np.cos(theta)), 
        n[0]*n[1]*(1-np.cos(theta)) - n[2]*np.sin(theta),
        n[0]*n[2]*(1-np.cos(theta)) + n[1]*np.sin(theta)
    ], [
        n[1]*n[0]*(1-np.cos(theta)) + n[2]*np.sin(theta),
        np.cos(theta) + n[1]**2*(1-np.cos(theta)),
        n[1]*n[2]*(1-np.cos(theta)) - n[0]*np.sin(theta),
    ], [
        n[2]*n[0]*(1-np.cos(theta)) - n[1]*np.sin(theta),
        n[2]*n[1]*(1-np.cos(theta)) + n[0]*np.sin(theta),
        np.cos(theta) + n[2]**2*(1-np.cos(theta))
    ]])
    
    return R

def _normalize_world_keypoints(wkps, rotate=True):
    body_parts = wkps.keys()
    
    # translate keypoints such that tail notch is at origin
    translated_wkps = {bp: wkps[bp] - wkps['HYPURAL_PLATE'] for bp in body_parts}

    if not rotate:
        return translated_wkps
    
    # perform first rotation
    u_base=np.array([1, 0, 0])
    v = translated_wkps['UPPER_LIP']
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps_intermediate = {bp: np.dot(R, translated_wkps[bp].T) for bp in body_parts}
    
    # perform second rotation
    u_base = np.array([0, 0, 1])
    v = norm_wkps_intermediate['ADIPOSE_FIN'] - np.array([norm_wkps_intermediate['ADIPOSE_FIN'][0], 0, 0])
    R = _generate_rotation_matrix(u_base, v)
    norm_wkps = {bp: np.dot(R, norm_wkps_intermediate[bp]) for bp in body_parts}
    
    # perform reflecton if necessary
    if norm_wkps['PECTORAL_FIN'][1] > 0:
        norm_wkps = {bp: np.array([
            norm_wkps[bp][0],
            -norm_wkps[bp][1],
            norm_wkps[bp][2]
        ]) for bp in body_parts}
    
    return norm_wkps

In [ ]:
wkps = {}
wkps['BODY'] = np.array(body_wkps)



In [ ]:
%matplotlib notebook
body_parts = [k for k in norm_wkps.keys() if k != 'BODY']
xs = [norm_wkps[bp][0] for bp in body_parts]
ys = [norm_wkps[bp][1] for bp in body_parts]
zs = [norm_wkps[bp][2] for bp in body_parts]
xs.extend(list(norm_wkps['BODY'][0]))
ys.extend(list(norm_wkps['BODY'][1]))
zs.extend(list(norm_wkps['BODY'][2]))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim3d(0, max(xs))
ax.set_ylim3d(-0.3, 0.3)
ax.set_zlim3d(-0.3, 0.3)
ax.scatter(xs, ys, zs, color='blue')

In [ ]:
%matplotlib notebook
xs = list(wkps['BODY'][:, 0])
ys = list(wkps['BODY'][:, 1])
zs = list(wkps['BODY'][:, 2])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# ax.set_xlim3d(0, max(xs))
# ax.set_ylim3d(-0.3, 0.3)
# ax.set_zlim3d(-0.3, 0.3)
ax.scatter(xs, ys, zs, color='blue')

<h1> Get Vikane Focusing Distance </h1>

In [ ]:
distances_from_camera = []
row_count = 0
for idx, row in df.iterrows():
    left_image_url, right_image_url = row.left_crop_url, row.right_crop_url
    if left_image_url is not None and right_image_url is not None:
        left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
        cm = row.camera_metadata

        left_image_f, _, _ = s3_access_utils.download_from_url(left_image_url)
        right_image_f, _, _ = s3_access_utils.download_from_url(right_image_url)
        imageL = cv2.imread(left_image_f)
        imageR = cv2.imread(right_image_f)
        H, kps = find_matches_and_homography(imageL, imageR, cm, left_crop_metadata, right_crop_metadata)
        wkps = get_wkps(kps, cm)
        dist = np.median([wkp[1] for wkp in wkps])
        distances_from_camera.append(dist)
        print('Row Count: {}, Number of Body Points: {}, Calculated Distance: {}'.format(row_count, len(kps), dist))

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.hist(distances_from_camera, bins=20)
plt.grid()
plt.xlabel('Distance from camera (m)')
plt.ylabel('Frequency')
plt.title('Distance distribution for accepted Vikane fish (pen_id=56)')
plt.show()

In [ ]:
distances_from_camera

In [ ]:
ps = [1/7, 3/7, 3/7]
pens = [0, 1, 2]

In [ ]:
N = 1000
e_x = ps[0]
var_x = ps[0] * (1 - ps[0])
std_x = np.sqrt(var_x)

In [ ]:
std_err = std_x / np.sqrt(N)

In [ ]:
np.array([e_x - 2.58 * std_err, e_x + 2.58 * std_err]) * N

In [ ]:
e_x * N

In [ ]:
means = []
trials = 1000
for t in range(trials):
    pen_list = []
    cum_ps = np.cumsum(ps)
    for n in range(N):
        r = np.random.uniform()
        pen = 1 if r < ps[0] else 0
        pen_list.append(pen)
    mean = np.mean(pen_list)
    means.append(mean)


In [ ]:
means = np.array(means)

In [ ]:
len(means[(means > e_x - 2.58 * std_err) & (means < e_x + 2.58 * std_err)])

In [ ]:
len(means)